# 이미지를 통한 얼굴 인식
1. 시스템에 입력된 이미지에서 얼굴 영역을 찾는다.(얼굴 검출, face detection)
2. 눈, 코, 입 등 얼굴의 특징점을 찾는다.(얼굴 정렬, face alignment)
3. 위 특징들을 통해 얼굴 영역을 동일한 형태와 크기로 변경(정규화, normalization)

# 영상을 통한 얼굴 인식


In [ ]:
입력된 이미지로 얼굴 인식
얼굴의 다른 부위(눈, 코 등) 인식
동영상에서 얼굴 인식
카메라로 얼굴 인식
표정 인식

카메라(영상)를 통한 표정 인식

In [1]:

!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2  908k    2 19194    0     0  47985      0  0:00:19 --:--:--  0:00:19 47865
 51  908k   51  464k    0     0   320k      0  0:00:02  0:00:01  0:00:01  320k
 69  908k   69  630k    0     0   261k      0  0:00:03  0:00:02  0:00:01  261k
 85  908k   85  776k    0     0   227k      0  0:00:03  0:00:03 --:--:--  227k
 97  908k   97  888k    0     0   196k      0  0:00:04  0:00:04 --:--:--  196k
100  908k  100  908k    0     0   200k      0  0:00:04  0:00:04 --:--:--  215k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3  878k    3 34137    0     0  38485      0  0:0

In [1]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


In [2]:
face_detection = cv2.CascadeClassifier('./files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('./files/emotion_model.hdf5', compile = False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

# camera객체 생성
camera = cv2.VideoCapture(0)

while True:
    # camera에서 이미지 캡쳐
    ret, frame = camera.read()
    # 이미지의 색상을 흑백으로 변환
    # cv2.cvtColor(원본 이미지, 색상 변환 코드)
    # HSV : Hue, Saturation, Value -> 색상, 채도, 명도
    
    
    # cv2.cvtColor(frame, cv2.COLOR_BRG2GRAY) -> cv2.imread(frame, cv2.IMREAD_GRAYSCALE)
    gray = cv2.imread(frame, cv2.IMREAD_GRAYSCALE)
    faces = face_detection.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
                                           minSize = (30, 30))
    canvas = np.zeros((250, 300, 3), dtype = 'uint8')
    
    if len(faces) > 0:
        face = sorted(faces, reverse = True, key = lambda x: (x[2] - x[0]) * 
                     (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
        cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
        
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            text = '{} : {:.2f}%'.formate(emotion, prob * 100)
            w = int(prob * 300)
            cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
            
        cv2.imshow('Emotion Recognition', frame)
        cv2.imshow('Probabilities', canvas)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): # 키보드 'q'를 누를때까지 대기
            break
            
camera.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
# cv2.imshow() : 그려진 파일을 보여준다.
# cv2.waitKey() : 키 입력이 있을 때까지 대기
# cv2.desoyAllWindows() : 윈도우창 종료

이미지를 통한 얼굴 인식
- 소프트맥스 손실 함수(sortmax loss function)

> 1) 교차 엔트로피(cross entropy)가 소프트 맥스 확률 분포와 정답 분포 사이의 오차를 계산

> 단점 : (1) 많은 데이터의 특징 공간을 효율적으로 학습하기 어렵다. (2) 전체적인 최적화를 고려하지 못하고 국부적인 최적 지점으로 쉽게 수렴할 수 있다. (3) 학습하지 않은 새로운 얼굴 이미지의 인식 성능이 낮아질 수 있다.

- 거리 기반 손실 함수(distance-based loss function)

> 대표적인 거리 기반 손실 함수 : 대비(contrastive), 트리플렛(triplet)

> 대비 손실 함수 : 두 얼굴 이미지의 쌍을 구성해 두 특징 벡터 간의 거리를 계산한다. 손실 값은 동일인의 두 벡터 간 거리가 멀거나, 비동일인의 두 벡터간 거리가 가까우면 커진다.

> 대비 손실 함수의 단점 : 동일인에 해당하는 두 벡터 거리와 비동일인의 두 벡터 거리가 개별적으로 학습된다.

> 트리플렛 손실 함수 : 범주가같은 벡터 간의 거리와 범주가 다른 벡터 간의 거리의 상대적 관계를 고려하는 방식으로 문제를 해결.
기준 이미지(a), 동일인 이미지(p), 비동일인 이미지(n)으로 구성된 트리플렛(a, p, n)을 구성 후, a와 p의 벡터 간 거리를 줄이는 동시에, a와 n의 벡터 간 거리를 넓혀서 거리차(|a-n| - |a-p|)가 개발자가 임의로 정한 마진값보다 크도록 한다. -> 대비 손실 함수로 학습된 인식 모델보다 더 높은 추론 성능을 보이는 경향이 있다.

> 트리프렛 손실 함수의 단점 : 얼굴 인식 관련 딥러닝 모델에서 거리 기반 손실 함수를 잘 사용하지 않는 추세이다. 트리플렛을 구성하는 방식에 따라 성능이 크게 달라진다. 특히 범주 수가 많을수록 효과적인 트리플렛을 구성하기가 어렵다. 컴퓨팅 비용도 많이 소모된다.
범주의 수가 수십 ~ 수백개 수준일 때 효과적이나, 수천 개 이상이면 잘 동작하지 않는다.


- 앵귤러 마진 기반 손실 함수(angular margin based loss function)

> 앵귤러 마진을 추가한 소프트 맥스 기반 손실 함수를 이용해 서로 다른 인물 간 거리를 충분히 넓히는 방향으로 진행되고 있는 추세. 모델의 일반화 성능이 높다.


''' https://tech.kakaoenterprise.com/63 '''

In [9]:
# !if not exist './files/haarcascade_eye.xml' mkdir files
!curl -L -o ./files/haarcascade_eye.xml https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 3024k    0 3024k    0     0  2392k      0 --:--:--  0:00:01 --:--:-- 2390k
100 3132k    0 3132k    0     0  2470k      0 --:--:--  0:00:01 --:--:-- 2468k


In [2]:
!curl -L -o C:/Users/sbs/Desktop/김동훈/Portfolio/files/haarcascade_frontalface_default.xml https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
!curl -L -o C:/Users/sbs/Desktop/김동훈/Portfolio/files/haarcascade_eye.xml https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 7138k    0 7138k    0     0  2896k      0 --:--:--  0:00:02 --:--:-- 2894k
100 8446k    0 8446k    0     0  3352k      0 --:--:--  0:00:02 --:--:-- 3351k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  480k    0  480k    0     0   521k      0 --:--:-- --:--:-- --:--:--  521k
100 3132k    0 3132k    0     0  2949k      0 --:--:--  0:00:01 --:--:-- 2949k


In [3]:
import cv2

# xml파일을 변수에 설정
face_cascade = cv2.CascadeClassifier('C:/Users/sbs/Desktop/김동훈/Portfolio/files/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('C:/Users/sbs/Desktop/김동훈/Portfolio/files/haarcascade_eye.xml')



src = cv2.imread('C:/Users/sbs/Desktop/김동훈/Portfolio/files/face.jpg')
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)



# detectMultiScale() : 얼굴을 인식해 검출하는 부분(a, b, c)
# a : 사진 정보, b : scaleFactor, 사진에서 얼굴이나 눈 등 지정한 형태를 찾을 정밀도, 기본값은 1.1
# c : minNeighbors, 얼굴 사이의 최소 간격, 기본값은 3 (b, c는 생략 가능)
faces = face_cascade.detectMultiScale(src_gray, 1.1, 3)
# 얼굴이 검출되면 (x, y, w, h)의 값을 가진 리스트 형태로 위치를 반환
# x, y : 검출된 얼굴의 좌상단 위치
# w, h : 가로, 세로 크기



for x, y, w, h in faces: # 얼굴
    # cv2.rectangle(a, b, c, d, e) -> 사각형
    # a : 이미지 정보
    # b : 좌측 상단 좌표
    # c : 우측 하단 좌표
    # d : brg색상
    # e : 사각형과 색상의 형태(두께). -1로 주면 사각형이 색으로 다 채워지고, 다른 양수값으로 넣으면 색이 다 채워지지 않은, 값에따라 변하는 두께의 사각형
    cv2.rectangle(src, (x, y), (x + w, y + h), (255, 0, 0), 2)
    face = src[y: y + h, x: x + w]
    face_gray = src_gray[y: y + h, x: x + w]
    eyes = eye_cascade.detectMultiScale(face_gray) # (face_gray, 1.1, 3) -> 1.1, 3은 기본값
    for (ex, ey, ew, eh) in eyes: # 눈
        cv2.rectangle(face, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

        
        
# cv2.imwrite (a, b). a : 저장될 파일명, b : 저장할 이미지
cv2.imwrite('C:/Users/sbs/Desktop/김동훈/Portfolio/files/face_cascade.jpg', src)





'''https://ponyozzang.tistory.com/597 참조'''

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# cv2.line(img, start, end, color, thickness)
# cv2.circle(img, center, radian, color, thickness)
# cv2.rectangle(img, start, end, color, thickness)
# cv2.ellipse(img, center, axes, angle, startAngle, endAngle, color, thickness)
# cv2.polylines(imge, pts, isClosed, color, thickness) -> 다각형
# cv2.putText(img, text, org, font, fontScale, color) -> 텍스트 삽입. org : 문자열의 위치

# thickness : pixel값으로 int를 넣어준다.

In [ ]:
'''참고 자료'''


'''
https://m.blog.naver.com/roboholic84/221633210887 - 딥러닝으로 표정 인식하기 알고리즘
https://tech.kakaoenterprise.com/63 - 얼굴 인식 알고리즘 선행 연구
http://www.kibme.org/resources/journal/20180801145420149.pdf - (논문) 딥러닝 기반 얼굴 검출,
랜드마크 검출 및 얼굴 인식 기술 연구 동향
https://openaccess.thecvf.com/content_CVPR_2020/papers/Kim_GroupFace_Learning_Latent_Groups_and_Constructing_Group-Based_Representations_for_Face_CVPR_2020_paper.pdf - AI Lab 논문
https://ukayzm.github.io/python-face-recognition/ - face_recognition을 활용한 얼굴 인식
http://suyeongpark.me/archives/6062 - OpenCV 4로 배우는 컴퓨터 비전과 머신 러닝/ 객체 검출



'''